# &#9678; Data Cleaning Project - Outlook Contacts


### Importing:

In [1]:
import pandas as pd
import numpy as np

In [2]:
joel = pd.read_csv("Joel_Hotmail.csv",encoding='Latin-1',dtype='object')

###### We will be creating a custom column called 'Contact'. This field consists of the first and last name attributes. This will make it easier to identify duplicate contacts. The customer says his not concerned with the middle name. But if that was the case, I would add the 'Middle Name' field into our concatenation.

In [3]:
joel["Contact"] = joel["First Name"].fillna('') +' '+ joel["Last Name"].fillna('')

____

###### Identifying the fields which contains at least one entry:

In [4]:
joel.dropna(axis=1,thresh=1).columns

Index(['Title', 'First Name', 'Middle Name', 'Last Name', 'Suffix', 'Company',
       'Department', 'Job Title', 'Business Street', 'Business City',
       'Business State', 'Business Postal Code', 'Business Country/Region',
       'Home Street', 'Home City', 'Home State', 'Home Postal Code',
       'Home Country/Region', 'Other Street', 'Other City', 'Other State',
       'Other Postal Code', 'Other Country/Region', 'Business Fax',
       'Business Phone', 'Business Phone 2', 'Car Phone', 'Company Main Phone',
       'Home Fax', 'Home Phone', 'Home Phone 2', 'Mobile Phone', 'Other Phone',
       'Pager', 'Primary Phone', 'Anniversary', 'Birthday', 'Children',
       'E-mail Address', 'E-mail Type', 'E-mail Display Name',
       'E-mail 2 Address', 'E-mail 2 Type', 'E-mail 2 Display Name',
       'E-mail 3 Address', 'E-mail 3 Type', 'E-mail 3 Display Name', 'Gender',
       'Initials', 'Notes', 'Office Location', 'Priority', 'Private',
       'Sensitivity', 'Spouse', 'Web Page', 'Conta

######  Stored on the variable 'imp' will be the names of the columns which are considered 'important'.

In [5]:
imp = list(['Contact','Business Phone','Business Phone 2','Home Phone','Home Phone 2','Mobile Phone','Other Phone',
            'E-mail Address','E-mail 2 Address', 'E-mail 3 Address'])

###### 'imp2' will contain the same elements found on 'imp' with the addition of the 'Notes' field.

In [6]:
imp2=list(['Contact','Business Phone','Business Phone 2','Home Phone','Home Phone 2','Mobile Phone','Other Phone',
            'E-mail Address','E-mail 2 Address', 'E-mail 3 Address','Notes'])

###### NOTE: The 'Notes' field is of utter importance for our client, since it contains meetings histories, comments, and sometimes even phone numbers and addresses (we will be dealing with these cases on section 1)

___

##### &#9678;  Initial number of Records:

In [7]:
#Initial Number of Records:
len(joel.index)

12155

##### &#9678; Initial Number of Duplicates on Contact

In [8]:
len(joel[joel.duplicated(subset=['Contact'],keep=False)])

3613

In [9]:
len(joel[joel.duplicated(['First Name','Last Name'],keep=False)])

3613

___

## 1. Preliminary cleaning: Cleaning 'Notes' Field

###### Before we start, there are some unwanted data on the 'Notes' field that we wish to get rid of. We're not actually deleting then, just turning them to blank notes so it doesn't mess up with our counting of nulled cells methods later on. It is important that we do this right at the beggining so that later they don't get merged when we jump to the aggregate operations. 

In [10]:
test = joel[joel['Notes'].str.contains("Contact Imported")==True]

###### Looking through the 'Notes' field I came across some notes like this one below:

In [11]:
# Example: Contact 329
test.loc[329]['Notes']

'-----------------------------\nContact Imported:\nEmailAddress : msanchez@rtvamerica.com\nBusinessPhone : 7862206482\nMobilePhone : 2024685300\nLine1 : 175 SW 7th Street \nSuite 1508\nCity : Miami\nState : FL\nPostalCode : 33130'

###### We can see that this entry has some valuable information contained within them like Business Phones, Addresses, et cetera... Information which,in their turn, are not contained by the fields which they were supposed to be,within the record itself:

In [12]:
# Example: Contact 329
test.loc[329][imp2]

Contact                                               Marcelo Sanchez
Business Phone                                                    NaN
Business Phone 2                                                  NaN
Home Phone                                                        NaN
Home Phone 2                                                      NaN
Mobile Phone                                                      NaN
Other Phone                                                       NaN
E-mail Address                                                    NaN
E-mail 2 Address                                                  NaN
E-mail 3 Address                                                  NaN
Notes               -----------------------------\nContact Importe...
Name: 329, dtype: object

In [13]:
## Non-null entries for Business Phone on this sub-dataframe
test['Business Phone'].count()  

0

###### Our objective here is to fix this by extracting only the useful information from these notes.

###### First, we are going to break those lenghty strings into small elements and comprised them into lists. We're going to split them by the '\n' substring. This is a special character used to describe a new line in Unix/MacOS

In [14]:
l = test.loc[:,'Notes'].apply(lambda x: x.split('\n'))

In [15]:
# Example: Contact 329
l[329]

['-----------------------------',
 'Contact Imported:',
 'EmailAddress : msanchez@rtvamerica.com',
 'BusinessPhone : 7862206482',
 'MobilePhone : 2024685300',
 'Line1 : 175 SW 7th Street ',
 'Suite 1508',
 'City : Miami',
 'State : FL',
 'PostalCode : 33130']

###### Next, lets separate the cream from the crop. All the useful data contained within these notes are characterized by the presence of the '  : ' substring. Lets filter out our lists so that it retains only the elements that contain those characters:

In [16]:
def WhatMatters(l):
    list1 = []
    for i in l:
        if i.find(' : ') != -1:
            list1.append(i)
    return list1

In [17]:
l = l.apply(WhatMatters)
l.loc[329]

['EmailAddress : msanchez@rtvamerica.com',
 'BusinessPhone : 7862206482',
 'MobilePhone : 2024685300',
 'Line1 : 175 SW 7th Street ',
 'City : Miami',
 'State : FL',
 'PostalCode : 33130']

###### Now, lets format the names of the variables in question so that they match the labels of our main dataframe. For example: At the notes example above, we can see the we have the label  'Line1' followed by a colon and the information about a person's Address. In the original dataset, however, the field which contains this kind of data is called 'Business Street'. That's why we will be creating and applying a function called 'FitNames', which will be formatting the labels inside our lists and replacing them with the ones that match our dataset:

In [18]:
def FitNames(list1):
    for i in range(0,len(list1)):
        if list1[i].find("Phone ") != -1:
            list1[i] = list1[i][:list1[i].find("P")] + ' ' + list1[i][list1[i].find("P"):]
        elif list1[i].find('Email') != -1:
            list1[i] = list1[i][:list1[i].find('mail')] + '-' + list1[i][list1[i].find('mail'):list1[i].find('A')] + ' '+ list1[i][list1[i].find('A'):]
        elif list1[i].find('City') != -1:
            list1[i] = 'Business' + ' ' + list1[i][list1[i].find('City'):]
        elif list1[i].find('State :') != -1:
            list1[i] = 'Business' + ' ' + list1[i][list1[i].find('State'):]
        elif list1[i].find('Line1') != -1:
            list1[i] = 'Business Street' + list1[i][list1[i].find(' '):]
        elif list1[i].find('PostalCode') != -1:
            list1[i] = 'Business Postal Code' + list1[i][list1[i].find(' '):]
        elif list1[i].find('Country :') != -1:
            list1[i] = 'Home Country/Region' + ' '+ list1[i][list1[i].find(':'):]
        
    return list1

In [19]:
l.apply(FitNames)
# Example: Contact 329
l.loc[329]

['E-mail Address : msanchez@rtvamerica.com',
 'Business Phone : 7862206482',
 'Mobile Phone : 2024685300',
 'Business Street : 175 SW 7th Street ',
 'Business City : Miami',
 'Business State : FL',
 'Business Postal Code : 33130']

In [20]:
l = l.apply(WhatMatters)

In [21]:
# Example: Contact 329
l.loc[329]

['E-mail Address : msanchez@rtvamerica.com',
 'Business Phone : 7862206482',
 'Mobile Phone : 2024685300',
 'Business Street : 175 SW 7th Street ',
 'Business City : Miami',
 'Business State : FL',
 'Business Postal Code : 33130']

###### With our lists finally cleaned up and containing only relevant and relatively organized information, we're ready to proceed by converting such lists into dictionaries objects...

In [22]:
def Dictionaries(list1):
    d={}
    temp =[]
    for i in list1:
        temp=i.split(' : ')
        d[temp[0]]=temp[1]
    return d

In [23]:
l  =l.apply(Dictionaries)

###### Once turned into dictionaries, we can easily treat our data like pandas Series as well!

In [24]:
# Example: Contact 329
pd.Series(l.loc[329])

E-mail Address          msanchez@rtvamerica.com
Business Phone                       7862206482
Mobile Phone                         2024685300
Business Street              175 SW 7th Street 
Business City                             Miami
Business State                               FL
Business Postal Code                      33130
dtype: object

In [25]:
len(l)

490

###### Finally, we just need to run one last command in order to merge our newly-discovered info. into our old dataframe. For that, we going to be using Pandas' built-in method called 'update'. This method will be vastly used throughout our cleaning process as it is much convenient to incorporate our changes into our target dataset.

In [26]:
# Example -Before-:
joel.loc[329][['Contact','Business Phone','Mobile Phone','Business Street','Business City','Business State',
              'Business Postal Code']]

Contact                 Marcelo Sanchez
Business Phone                      NaN
Mobile Phone                        NaN
Business Street                     NaN
Business City                       NaN
Business State                      NaN
Business Postal Code                NaN
Name: 329, dtype: object

In [27]:
for i in l.index:
    joel.loc[i].update(pd.Series(l[i]))

In [28]:
# Example -After-:
joel.loc[329][['Contact','Business Phone','Mobile Phone','Business Street','Business City','Business State',
              'Business Postal Code']]

Contact                    Marcelo Sanchez
Business Phone                  7862206482
Mobile Phone                    2024685300
Business Street         175 SW 7th Street 
Business City                        Miami
Business State                          FL
Business Postal Code                 33130
Name: 329, dtype: object

###### Now that we've extracted the useful contact information out of the notes containing the 'Contact Imported' substring, we're going to go ahead and turn them into blank:

In [29]:
## Deleting Unwanted notes
joel.loc[joel['Notes'].str.contains("Contact Imported")==True,'Notes'] = ''

___

## 2. Records Missing Crucial Data:

###### Some records consist of just the contact name or no contact name at all. After speaking with the client, we decided that the best approach would be to just to delete them altogether.

In [30]:
#Getting rid of records with nothing but missing values on 'First Name','Middle Name' and 'Last Name' fields...
joel = joel.dropna(subset=['First Name','Middle Name','Last Name'],how='all')

In [31]:
#Getting rid of records with nothing but missing values on all other key fields...
joel = joel.dropna(subset=['Business Fax','Business Phone','Business Phone 2','Home Phone','Home Phone 2',
                  'Mobile Phone','Other Phone','E-mail Address','E-mail 2 Address','Company','Notes'],how='all')

##### &#9678; Current Number of Records:

In [32]:
len(joel.index)

9802

---

## 3. Converging Data:

###### We will be now trying to condense our data inside our dataframe. We wil be moving information from relativaly 'less' important columns like 'Home Phone 2', 'Business Phone 2'  into their respective main field (in this case: 'Home Phone' and 'Business Phone'). However, we will be moving only if theres no data already sitting at the target attribute, to avoid losing info.:

In [33]:
## Moving phone data into primary (target) field if there's space:
transfer =joel[joel['Home Phone 2'].notnull() & (joel['Home Phone'].isnull())].index
joel.loc[transfer,'Home Phone'] = joel.loc[transfer]['Home Phone 2']
# Deleting info from source attribute:
joel.loc[transfer,'Home Phone 2'] = np.nan

In [34]:
## Moving phone data into primary (target) field if there's space:
transfer =joel[joel['Business Phone 2'].notnull() & (joel['Business Phone'].isnull())].index
joel.loc[transfer,'Business Phone'] = joel.loc[transfer]['Business Phone 2']
# Deleting info from source attribute:
joel.loc[transfer,'Business Phone 2'] = np.nan

In [35]:
## Moving phone data into primary (target) field if there's space:
transfer =joel[joel['Home Phone 2'].notnull() & (joel['Other Phone'].isnull())].index
joel.loc[transfer,'Other Phone'] = joel.loc[transfer]['Home Phone 2']
# Deleting info from source attribute:
joel.loc[transfer,'Home Phone 2'] = np.nan

In [36]:
## Moving phone data into primary (target) field if there's space:
transfer =joel[joel['Other Phone'].notnull() & (joel['Mobile Phone'].isnull())].index
joel.loc[transfer,'Mobile Phone'] = joel.loc[transfer]['Other Phone']
# Deleting info from source attribute:
joel.loc[transfer,'Other Phone'] = np.nan

In [37]:
## Moving phone data into primary (target) field if there's space:
transfer =joel[joel['E-mail 2 Address'].notnull() & (joel['E-mail Address'].isnull())].index
joel.loc[transfer,'E-mail Address'] = joel.loc[transfer]['E-mail 2 Address']
# Deleting info from source attribute:
joel.loc[transfer,'E-mail 2 Address'] = np.nan

### 3.1 Creating Custom Columns:

###### Let's also create custom fields which concatenate phones and emails from each record so that it facilitates our job later on...

##### I left the counter of elements of the most inclusive column for each merged field I'm creating just to make sure I'm not leaving anything behind  or unchecked

In [38]:
joel['Home Phone'].count()

2437

In [39]:
#Creating Field 'Home Phones' for analysis' sake
joel["Home Phones"] = joel["Home Phone"].fillna('') +' '+ joel["Home Phone 2"].fillna('')

joel.loc[joel['Home Phones'] == ' ','Home Phones'] = np.nan

joel['Home Phones'].count()

2437

In [40]:
joel['Business Phone'].count()

2950

In [41]:
#Creating Field 'Business Phones' for analysis' sake
joel["Business Phones"] = joel["Business Phone"].fillna('') +' '+ joel["Business Phone 2"].fillna('')

joel.loc[joel['Business Phones'] == ' ','Business Phones'] = np.nan

joel['Business Phones'].count()

2950

In [42]:
joel['Mobile Phone'].count()

4513

In [43]:
#Creating Field 'Other Phones' for analysis' sake
joel["Other Phones"] = joel["Mobile Phone"].fillna('') +' '+ joel["Other Phone"].fillna('')

joel.loc[joel['Other Phones'] == ' ','Other Phones'] = np.nan

joel['Other Phones'].count()

4513

In [44]:
phones = list(['Contact','Business Phone','Business Phone 2','Home Phone','Home Phone 2','Mobile Phone','Other Phone'])

In [45]:
joel['E-mail Address'].count()

5588

In [46]:
##Creating Field 'E-mail Addresses' for analysis' sake
joel["E-mail Addresses"] = joel["E-mail Address"].fillna('') +' '+ joel["E-mail 2 Address"].fillna('')+' '+joel["E-mail 3 Address"].fillna('')

joel.loc[joel['E-mail Addresses'] == '  ','E-mail Addresses'] = np.nan

joel['E-mail Addresses'].count()

5588

## 4. Dealing with Duplicates:

##### &#9678; Current Number of Duplicates on Contact

In [47]:
len(joel[joel.duplicated(subset=['Contact'],keep=False)])

1421

____

###### Let's begin by subsetting our main dataframe into a smaller one containing only records which have the same 'Contact' field information:

In [48]:
duplicates = joel[joel.duplicated(subset=['Contact'],keep=False)].sort_values('Contact')

##### Next, let's reduce our subset even more! let's identify records with same (duplicated) information also on important fields:

In [49]:
duplicates=duplicates[duplicates.duplicated(subset=imp,keep=False)]

###### Now that we have taken the subsetted version of our dataframe containing only records with duplicated info on'Contact' AND on important fields besides 'Notes', we will converge all notes into the first occurrence and keep that occurence.

##### To do that, we are first going to find which columns need to be joined, which don't, as well as the ones which need to be aggregated differently: by keeping the first of the duplicate group.

In [50]:
# Finding columns which have at least five entries:
relevant = joel.dropna(axis=1,thresh=5).columns
relevant

Index(['Title', 'First Name', 'Middle Name', 'Last Name', 'Suffix', 'Company',
       'Department', 'Job Title', 'Business Street', 'Business City',
       'Business State', 'Business Postal Code', 'Business Country/Region',
       'Home Street', 'Home City', 'Home State', 'Home Postal Code',
       'Home Country/Region', 'Other Street', 'Other City', 'Other State',
       'Other Postal Code', 'Other Country/Region', 'Business Fax',
       'Business Phone', 'Business Phone 2', 'Car Phone', 'Company Main Phone',
       'Home Fax', 'Home Phone', 'Home Phone 2', 'Mobile Phone', 'Other Phone',
       'Pager', 'Anniversary', 'Birthday', 'Children', 'E-mail Address',
       'E-mail Type', 'E-mail Display Name', 'E-mail 2 Address',
       'E-mail 2 Type', 'E-mail 2 Display Name', 'E-mail 3 Address',
       'E-mail 3 Type', 'E-mail 3 Display Name', 'Gender', 'Initials', 'Notes',
       'Office Location', 'Priority', 'Private', 'Sensitivity', 'Spouse',
       'Web Page', 'Contact', 'Home Phones

In [51]:
# These are the fields which consist of duplicated information in our current subset.
# They won't need to be joined since they only contain redundant info. 
# They will be aggregated by deleting all occurences except by first inside each duplicate group.

firsts = ['Business Phone','Business Phone 2','Home Phone','Home Phone 2','Mobile Phone','Other Phone','E-mail Address',
'E-mail 2 Address','E-mail 3 Address']

In [52]:
# Reffering to next three commands: 
# The rest however will be aggregated by joining each entry into a single record, each entry will be separated by ' | '
# inside the cell
join_these = list(relevant[~relevant.isin(firsts)])

In [53]:
# These, however, won't be needed to be joined, either because of their content or because of determinations made by the 
# client

list_to_remove=['First Name', 'Middle Name', 'Last Name','Contact',
 'Home Phones',
 'Business Phones',
 'Other Phones',
 'E-mail Addresses','Anniversary','Birthday','E-mail Type', 
    'Gender','Priority ','Private ','Sensitivity','Priority','Private','E-mail Display Name','Initials']

In [54]:
join_these= list(set(join_these).difference(set(list_to_remove)))

In [55]:
## Lets create a python dictionary which will give directions to our .agg method as to which type of aggregation ('first' or 'join')
# should be used to each attribute. 

dicts = {}
value_1 = 'first'
value_2 = lambda x: ' | '.join(x.fillna('').astype(str))

for i in firsts:           
    dicts[i] = value_1

for j in join_these:
    dicts[j] = value_2

In [56]:
duplicates.reset_index(inplace=True)

dicts['index'] = 'first'

# The aggregation happens here! Let's cast our dictionary inside the .agg method so it does its magic!
#                                                    \/
a= duplicates.groupby('Contact',as_index=False).agg(dicts)
                    
a.set_index('index',inplace=True)

In [57]:
duplicates.set_index('index',inplace=True)

In [58]:
# duplcates Left JOIN a
duplicates.update(a, join='left', overwrite=True, filter_func=None, raise_conflict=False)

###### Let's take a look at our 'duplicates' dataframe

In [59]:
# EXAMPLE:
duplicates[duplicates.Contact == ' Center']

,Title,First Name,Middle Name,Last Name,Suffix,Company,Department,Job Title,Business Street,Business Street 2,...,User 1,User 2,User 3,User 4,Web Page,Contact,Home Phones,Business Phones,Other Phones,E-mail Addresses
index,,,,,,,,,,,,,,,,,,,,,
3943,| |,NaN,Service,Center,| |,U.S.C.I.S. | Immigration & Naturalization Serv...,| |,| |,"P.O. Box 87485\nLincoln, NE 68501-7485 | P.O. ...",NaN,...,NaN,NaN,NaN,NaN,| |,Center,NaN,NaN,NaN,NaN
4716,NaN,NaN,Forms,Center,NaN,Immigration & Naturalization Service,NaN,NaN,"P.O. Box 567\nWilliston, VT 05495-0567",NaN,...,NaN,NaN,NaN,NaN,NaN,Center,NaN,NaN,NaN,NaN
2671,NaN,NaN,Backlog Processing,Center,NaN,Employment & Training Administration,NaN,NaN,1 Belmont Ave.\nSuite 220,NaN,...,NaN,NaN,NaN,NaN,NaN,Center,NaN,NaN,NaN,NaN


###### Let's take the 'Company' field for instance:

In [60]:
duplicates.loc[3943]['Company']  # All info was joined to a single cell!

'U.S.C.I.S. | Immigration & Naturalization Service | Employment & Training Administration'

###### We can see that the data were joined, but it looks like we ended up with some unwanted '|  ' character. let's create a function that uses regular expressions to raise  'True' if a cell contains only '| ' character.

In [61]:
import re
>>> def special_match(strg, search=re.compile(r'[^| .]').search):
...     return not bool(search(strg))

###### Now lets get rid of those unnecessary entries by applying the following lambda expression:

In [62]:
duplicates = duplicates.applymap(lambda x: np.nan if (isinstance(x,str) and special_match(x) == True) else x)

In [63]:
# EXAMPLE:
duplicates[duplicates.Contact == ' Center'][join_these]

,Business City,Company,Title,Other Country/Region,Children,Pager,Business Fax,Office Location,Home State,E-mail 2 Type,...,Spouse,Business Postal Code,Other State,Home City,Business Country/Region,Notes,Home Country/Region,Home Fax,Other City,Suffix
index,,,,,,,,,,,,,,,,,,,,,
3943,| | Bala Cynwyd,U.S.C.I.S. | Immigration & Naturalization Serv...,NaN,NaN,NaN,NaN,| | +1 (484) 270-1615,NaN,NaN,NaN,...,NaN,| | 19004,NaN,NaN,NaN,"Sunday, September 15, 2013 7:20 PM:\nCenter, N...",NaN,NaN,NaN,NaN
4716,NaN,Immigration & Naturalization Service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 7:20 PM:\nCenter, E...",NaN,NaN,NaN,NaN
2671,Bala Cynwyd,Employment & Training Administration,NaN,NaN,NaN,NaN,+1 (484) 270-1615,NaN,NaN,NaN,...,NaN,19004,NaN,NaN,NaN,"Sunday, September 15, 2013 8:09 PM:\n<!DOCTYPE...",NaN,NaN,NaN,NaN


In [64]:
# Let's also create a lambda function that uses a regular expression in order to remove repeated character pattern 
# in a string (for preventing redundant info after the aggregation)
import re
duplicates[join_these] = duplicates[join_these].applymap(lambda x: re.sub(r'(.+?)\1+', r'\1', x) if isinstance(x,str)  else x)

____

###### OK! Now, let's see how we got rid of many duplicates like these ones:

In [65]:
## Example -Before- :
joel[joel['Contact']=='Tatiana Goncalves'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
721,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
722,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
723,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
724,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
725,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
726,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
727,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
728,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
729,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with Urbano.\r\n\r\n
730,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n


###### Let's use the '.update' method to carry our changes to our main df 'joel'

In [66]:
# test Left JOIN nodupes_phone
joel.update(duplicates, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [67]:
# Drop the duplicates using column 'Contact' as reference but only keep the most filled rows
duplicates_2= duplicates.drop_duplicates(subset=['Contact'],keep='first')

joel.drop(duplicates.index.difference(duplicates_2.index),inplace=True)

In [68]:
## Example -After- :
joel[joel['Contact']=='Tatiana Goncalves'][imp2]


,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
726,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n | Works with\r\n\r\n | Work...


_____

##### &#9678; Current Number of Duplicates on 'Contact':

In [69]:
len(joel[joel.duplicated(subset=['Contact'],keep=False)])

926

____

##### As I began a deeper exploration of this dataset, I came across lots of almost identical duplicated records. That is, they had similar data on Contact names but presented divergent information on other fields such as E-mails, Business Phones, Home Phones and Mobile Phone. In the following subsections, we will be performing a series of methods to promote condensation of inforrmation into a single record within a group of 'duplicated' data.

#### Typical Example of this phenomena:

In [70]:
joel[joel.Contact.str.contains('Matthew Adams')][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
5529,Matthew Adams,(561) 272-8037,NaN,NaN,NaN,+1 (561) 654-6288,NaN,mattadams97@hotmail.com,NaN,NaN,"Sunday, September 15, 2013 7:19 PM:\nMatthew D..."
6406,Matthew Adams,NaN,NaN,NaN,NaN,(561) 542-5720,NaN,mattadams14@gmail.com,NaN,NaN,NaN


### 4.1 Records with non-duplicated Mails and with duplicated name:

In [71]:
nodupes_mail = joel[((~joel.duplicated(subset=['E-mail Address'], keep='last')) |(~joel.duplicated(subset=['E-mail Address'], keep='first')) | (joel['E-mail Address'].isnull())) &((joel.duplicated(subset=['Contact'],keep=False)))]


### 4.1.1  Dealing with Pairs

#### After identifying the overall redundant Names, we're going to first filter out those records which have only pairs of duplictates, so that we can move not similar e-mails in order to condense all information into a single record.


In [72]:
pair = nodupes_mail.groupby(nodupes_mail['Contact'].values.tolist(),as_index=False).size() ==2
lista = list(pair[pair==True].index)


# Function to identify pairs of duplicated records
def pairs(x):
    if x in lista:
        return True

In [73]:
# Applying the function into the 'Contact' column and subsequently filtering our current subset:

nodupes_mail = nodupes_mail[nodupes_mail['Contact'].apply(pairs) == True]

###### Since we now have only pairs of records which have duplicated names and different e-mail addresses, we can safely move data from 'E-mail' to 'E-mail 2'.

In [74]:
## Moving those e-mails From 'E-mail' Field to 'E-mail 2' field

nodupes_mail['E-mail 2 Address'] = nodupes_mail['E-mail Address'][nodupes_mail.duplicated(subset=
['Contact'], keep=False)]

###### We're taking the approach of sorting by a 'count' column which counts the number of cells with no information on importante fields (imp2) for each row. That way, we make sure we're merging information into the most filled row of each duplicate group.

In [75]:
#sorting:
nodupes_mail['count'] = pd.isnull(nodupes_mail[imp2]).sum(1)
nodupes_mail= nodupes_mail.sort_values(['Contact','count','E-mail Address'],na_position='last')


###### Now we're going to go ahead and use the shift method to move up one cell on the E-mail 2 Address. That way we get the second address into our first record, which is the most filled one since we previously sorted by our 'count' column.

In [76]:
## Part 2.Moving  those e-mails From 'E-mail 2 Address' to the upper records (same Person)
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!

nodupes_mail['E-mail 2 Address'] = nodupes_mail.groupby(['Contact'])['E-mail 2 Address'].shift(-1)

___

#### Joining back to original dataset:

In [77]:
#test -Before-:
joel[joel['Contact']=='Miguel de Oliveira'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
24,Miguel de Oliveira,NaN,NaN,NaN,NaN,(407) 467-8843,NaN,maoliveira@earthlink.net,NaN,NaN,Amigo de Joey Volpato\r\n\r\nCCBTV\r\n\r\nNot ...
25,Miguel de Oliveira,NaN,NaN,NaN,NaN,01155966451901,NaN,miguel1956@uol.com.br,NaN,NaN,Olivia Ilmar Dantas da Silva Oliveira\n\nVeiu ...


In [78]:
# act Left JOIN nodupes_phone
joel.update(nodupes_mail, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [79]:
#test -After-: (We are currently concerned only with E-mail addresses)
joel[joel['Contact']=='Miguel de Oliveira'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
24,Miguel de Oliveira,NaN,NaN,NaN,NaN,(407) 467-8843,NaN,maoliveira@earthlink.net,miguel1956@uol.com.br,NaN,Amigo de Joey Volpato\r\n\r\nCCBTV\r\n\r\nNot ...
25,Miguel de Oliveira,NaN,NaN,NaN,NaN,01155966451901,NaN,miguel1956@uol.com.br,NaN,NaN,Olivia Ilmar Dantas da Silva Oliveira\n\nVeiu ...


### 4.1.2 Dealing with trios

###### Now we're going to be dealing with trios of redundant records (on Contact name). I've taken this approach due to limitations of Outlook itself as it only offers three fields for E-mails. For bigger groups, We will be dealing with them on the later sections...

In [80]:
nodupes_mail = joel[((~joel.duplicated(subset=['E-mail Addresses'], keep='last')) | (~joel.duplicated(subset=['E-mail Addresses'], keep='first')) | (joel['E-mail Addresses'].isnull()))  &((joel.duplicated(subset=['Contact'],keep=False)))]


In [81]:
pair = nodupes_mail.groupby(nodupes_mail['Contact'].tolist(),as_index=False).size() ==3
lista = list(pair[pair==True].index)
nodupes_mail = nodupes_mail[nodupes_mail['Contact'].apply(pairs) == True]

----

###### We're going to be pretty much taking the same steps that we've taken for dealing with duplicates. The only difference being that now we have to include 'E-mail 3 Address' into the equation!

In [82]:
## Moving those emails From 'E-mail Address' Field to 'E-mail 2 Address' field
nodupes_mail['E-mail 2 Address'] = nodupes_mail['E-mail Address'][nodupes_mail.duplicated(subset=
['Contact'], keep=False)]
## Moving those emails From 'E-mail 2 Address' Field to 'E-mail 3 Address' field
nodupes_mail['E-mail 3 Address'] = nodupes_mail['E-mail 2 Address']
#sorting:
nodupes_mail['count'] = pd.isnull(nodupes_mail[imp2]).sum(1)
nodupes_mail= nodupes_mail.sort_values(['Contact','count','E-mail Address'],na_position='last')
nodupes_mail.drop('count',1,inplace=True)
#Shifting Values:
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_mail['E-mail 2 Address'] = nodupes_mail.groupby(['Contact'])['E-mail 2 Address'].shift(-1)
nodupes_mail['E-mail 3 Address'] = nodupes_mail.groupby(['Contact'])['E-mail 3 Address'].shift(-2)

#### Joining back to original dataset:

In [83]:
#test -Before-:
joel[joel['Contact']=='Selma Smith'][imp]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address
8758,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,Alohaselma@yahoo.com,NaN,NaN
10342,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,selfiorini@msn.com,NaN,NaN
11465,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,selsmith66@yahoo.com,NaN,NaN


In [84]:
# act Left JOIN nodupes_mail
joel.update(nodupes_mail, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [85]:
#test -After-:
joel[joel['Contact']=='Selma Smith'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
8758,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,Alohaselma@yahoo.com,selfiorini@msn.com,selsmith66@yahoo.com,NaN
10342,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,selfiorini@msn.com,selsmith66@yahoo.com,NaN,NaN
11465,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,selsmith66@yahoo.com,NaN,NaN,NaN


### 4.2 Records with non-duplicated Phones and with duplicated name:

###### Now we're going to be doing the same but with phone information. This will be a long subsection since we have many attributes for phone numbers:

In [86]:
phones

['Contact',
 'Business Phone',
 'Business Phone 2',
 'Home Phone',
 'Home Phone 2',
 'Mobile Phone',
 'Other Phone']

### 4.2.1  Dealing with Pairs

In [87]:
# Function to identify pairs of duplicated records
def pairs(x):
    if x in lista:
        return True

# EYE - jack of all trades

#### Due to the particularities of the dataset, we will be merging non-similar phone information of paired duplicates by moving and spreading data into the corresponding pair of each phone field in a single contact record. 
##### Each phone field has its correspondent equivalent for a second entry, for instance: 'Business Phone' has 'Business Phone 2', 'Home Phone' has 'Home Phone 2', 'Mobile Phone' has 'Other Phone'. 'Other Phone' will be treated as a "jack-of-all-trades" type of field".
##### Examples will be provided all the way through so this doesn't get overwhelming.

#### Business Phones

In [88]:
nodupes_phone_bus = joel[((~joel.duplicated(subset=['Business Phones'], keep='last'))  |(~joel.duplicated(subset=['Business Phones'], keep='first')) | (joel['Business Phones'].isnull())) & ((joel.duplicated(subset=['Contact'],keep=False)))]

In [89]:
pair = nodupes_phone_bus.groupby(nodupes_phone_bus['Contact'].tolist(),as_index=False).size() ==2
lista = list(pair[pair==True].index)
nodupes_phone_bus = nodupes_phone_bus[nodupes_phone_bus['Contact'].apply(pairs) == True]

In [90]:
## Moving those Phones From 'Business Phone' Field to 'Business Phone 2' field
nodupes_phone_bus['Business Phone 2'] = nodupes_phone_bus['Business Phone'][nodupes_phone_bus.duplicated(subset=
['Contact'], keep=False)]

###### We're taking the approach of sorting by a 'count' column which counts the number of cells with no information for each row. That way, we make sure we're merging information into the most filled row of each duplicate group

In [91]:
#sorting:
nodupes_phone_bus['count'] = pd.isnull(nodupes_phone_bus[imp2]).sum(1)
nodupes_phone_bus= nodupes_phone_bus.sort_values(['Contact','count','Business Phone'],na_position='last')


In [92]:
## Part 2.Moving  those Phones From 'M-Phone' to the upper records (same Person)
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_phone_bus['Business Phone 2'] = nodupes_phone_bus.groupby(['Contact'])['Business Phone 2'].shift(-1)

___

# EYE

In [93]:
#Example -before-:
joel[joel['Contact']==' Bezerra'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
2604,Bezerra,(727) 216-7979,NaN,+1 (727) 812-8874,NaN,NaN,NaN
3566,Bezerra,(305) 254-6672,NaN,(305) 254-6672,NaN,(305) 281-8691,+1 (305) 254-6672


In [94]:
# joel Left JOIN nodupes_phone_bus
joel.update(nodupes_phone_bus, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [95]:
#Example -after- (concerned only with 'Business Phone' at the moment):
joel[joel['Contact']==' Bezerra'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
2604,Bezerra,(727) 216-7979,NaN,+1 (727) 812-8874,NaN,NaN,NaN
3566,Bezerra,(305) 254-6672,(727) 216-7979,(305) 254-6672,NaN,(305) 281-8691,+1 (305) 254-6672


___

#### Home Phones

In [96]:
nodupes_phone_hom = joel[((~joel.duplicated(subset=['Home Phones'], keep='last')) | (~joel.duplicated(subset=['Home Phones'], keep='first')) | (joel['Home Phones'].isnull())) & ((joel.duplicated(subset=['Contact'],keep=False)))]

In [97]:
pair = nodupes_phone_hom.groupby(nodupes_phone_hom['Contact'].tolist(),as_index=False).size() ==2
lista = list(pair[pair==True].index)
nodupes_phone_hom = nodupes_phone_hom[nodupes_phone_hom['Contact'].apply(pairs) == True]

In [98]:
## Moving those Phones From 'Home Phone' Field to 'Home Phone 2' field
nodupes_phone_hom['Home Phone 2'] = nodupes_phone_hom['Home Phone'][nodupes_phone_hom.duplicated(subset=
['Contact'], keep=False)]

In [99]:
#sorting:
nodupes_phone_hom['count'] = pd.isnull(nodupes_phone_hom[imp2]).sum(1)
nodupes_phone_hom= nodupes_phone_hom.sort_values(['Contact','count','Home Phone'],na_position='last')


In [100]:
## Part 2.Moving  those Phones From 'M-Phone' to the upper records (same Person)
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_phone_hom['Home Phone 2'] = nodupes_phone_hom.groupby(['Contact'])['Home Phone 2'].shift(-1)

___

In [101]:
#Example -before-:
joel[joel['Contact']=='Victor Hugo'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
5427,Victor Hugo,NaN,NaN,4075484101,NaN,NaN,NaN
6055,Victor Hugo,NaN,NaN,+55 24 99925-5410,NaN,NaN,NaN


In [102]:
# joel Left JOIN nodupes_phone_hom
joel.update(nodupes_phone_hom, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [103]:
#Example -after-:
joel[joel['Contact']=='Victor Hugo'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
5427,Victor Hugo,NaN,NaN,4075484101,+55 24 99925-5410,NaN,NaN
6055,Victor Hugo,NaN,NaN,+55 24 99925-5410,NaN,NaN,NaN


___


#### Mobile and Other Phones

In [104]:
nodupes_phone_other = joel[((~joel.duplicated(subset=['Other Phones'], keep='last')) | (~joel.duplicated(subset=['Other Phones'], keep='first')) | (joel['Other Phones'].isnull())) & ((joel.duplicated(subset=['Contact'],keep=False)))]

In [105]:
pair = nodupes_phone_other.groupby(nodupes_phone_other['Contact'].tolist(),as_index=False).size() ==2
lista = list(pair[pair==True].index)
nodupes_phone_other = nodupes_phone_other[nodupes_phone_other['Contact'].apply(pairs) == True]

In [106]:
## Moving those Phones From 'Mobile Phone' Field to 'Other Phone' field
nodupes_phone_other['Other Phone'] = nodupes_phone_other['Mobile Phone'][nodupes_phone_other.duplicated(subset=
['Contact'], keep=False)]

In [107]:
#sorting:
nodupes_phone_other['count'] = pd.isnull(nodupes_phone_other[imp2]).sum(1)
nodupes_phone_other= nodupes_phone_other.sort_values(['Contact','count','Mobile Phone'],na_position='last')


In [108]:
## Part 2.Moving  those Phones From 'M-Phone' to the upper records (same Person)
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_phone_other['Other Phone'] = nodupes_phone_other.groupby(['Contact'])['Other Phone'].shift(-1)

___

In [109]:
#Example -Before-:
joel[joel['Contact']=='Maria Abreu'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
706,Maria Abreu,NaN,NaN,NaN,NaN,(786) 599-7994,NaN
7685,Maria Abreu,NaN,NaN,NaN,NaN,+1 (407) 963-8598,NaN


In [110]:
# joel Left JOIN nodupes_phone_other
joel.update(nodupes_phone_other, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [111]:
#Example -After-:
joel[joel['Contact']=='Maria Abreu'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
706,Maria Abreu,NaN,NaN,NaN,NaN,(786) 599-7994,+1 (407) 963-8598
7685,Maria Abreu,NaN,NaN,NaN,NaN,+1 (407) 963-8598,NaN


#### Result of complete condensing:

#### We will be reviewing other complete condensing examples at the end.

In [112]:
#Example -after-:
joel[joel['Contact']==' Bezerra'][phones]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone
2604,Bezerra,(727) 216-7979,NaN,+1 (727) 812-8874,NaN,NaN,NaN
3566,Bezerra,(305) 254-6672,(727) 216-7979,(305) 254-6672,+1 (727) 812-8874,(305) 281-8691,+1 (305) 254-6672


___

##### In order to accomodate trios of different numbers for redundant contacts, we're going to merge the numbers into trios of non-conflicted fields. It will be the same as the pairs approach but with the addition of dumping the third phone into the 'Other Phone' attribute. Save for the last field 'Mobile Phone'. On this one we're going to have to improvise! We're going to move the second number into 'Other Phone' and then we will be dumping the third phone into 'Business 2 Phone' 

### 4.2.2 Exceptions: Dealing with trios

#### Business Phone to Business Phone 2 and Other Phone

In [113]:
nodupes_phone_bus = joel[((~joel.duplicated(subset=['Business Phones'], keep='last')) | (~joel.duplicated(subset=['Business Phones'], keep='first')) | (joel['Business Phones'].isnull())) & ((joel.duplicated(subset=['Contact'],keep=False)))]

In [114]:
# Function to identify pairs of duplicated records
def pairs(x):
    if x in lista:
        return True

In [115]:
pair = nodupes_phone_bus.groupby(nodupes_phone_bus['Contact'].tolist(),as_index=False).size() ==3
lista = list(pair[pair==True].index)
nodupes_phone_bus = nodupes_phone_bus[nodupes_phone_bus['Contact'].apply(pairs) == True]

___

In [116]:
## Moving those Phones From 'Business Phone' Field to 'Business Phone 2' field
nodupes_phone_bus['Business Phone 2'] = nodupes_phone_bus['Business Phone'][nodupes_phone_bus.duplicated(subset=
['Contact'], keep=False)]
## Moving those Phones From 'Business Phone 2' Field to 'Other Phone' field
nodupes_phone_bus['Other Phone'] = nodupes_phone_bus['Business Phone 2']
#sorting:
nodupes_phone_bus['count'] = pd.isnull(nodupes_phone_bus[imp2]).sum(1)
nodupes_phone_bus= nodupes_phone_bus.sort_values(['Contact','count','Business Phone'],na_position='last')
nodupes_phone_bus.drop('count',1,inplace=True)
#Shifting Values:
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_phone_bus['Business Phone 2'] = nodupes_phone_bus.groupby(['Contact'])['Business Phone 2'].shift(-1)
nodupes_phone_bus['Other Phone'] = nodupes_phone_bus.groupby(['Contact'])['Other Phone'].shift(-2)

#### Joining back to original Data set and deleting less useful rows:

In [117]:
#test -Before-:
joel[joel['Contact']==' Arruda'][['Contact','Business Phone',
                                'Business Phone 2','Other Phone','E-mail Address','E-mail 2 Address','E-mail 3 Address']]

,Contact,Business Phone,Business Phone 2,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address
2167,Arruda,(305) 801-9035,NaN,NaN,arruda@itamaraty.gov.br,u27comandante@yahoo.com.br,fernando.arruda@gmail.com
2642,Arruda,(305) 285-6222,NaN,NaN,fernando.arruda@itamaraty.gov.br,arruda@itamaraty.gov.br,u27comandante@yahoo.com.br
4872,Arruda,21 21047256,NaN,NaN,u27comandante@yahoo.com.br,NaN,NaN


In [118]:
# joel Left JOIN nodupes_phone_bus
joel.update(nodupes_phone_bus, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [119]:
#test -After-:
joel[joel['Contact']==' Arruda'][['Contact','Business Phone',
                                    'Business Phone 2','Other Phone','E-mail Address','E-mail 2 Address','E-mail 3 Address']]

,Contact,Business Phone,Business Phone 2,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address
2167,Arruda,(305) 801-9035,21 21047256,NaN,arruda@itamaraty.gov.br,u27comandante@yahoo.com.br,fernando.arruda@gmail.com
2642,Arruda,(305) 285-6222,(305) 801-9035,21 21047256,fernando.arruda@itamaraty.gov.br,arruda@itamaraty.gov.br,u27comandante@yahoo.com.br
4872,Arruda,21 21047256,NaN,NaN,u27comandante@yahoo.com.br,NaN,NaN


---

#### Home Phone to Home Phone 2 and Other Phone

In [120]:
nodupes_phone_hom = joel[((~joel.duplicated(subset=['Home Phones'], keep='last')) | (~joel.duplicated(subset=['Home Phones'], keep='first')) | (joel['Home Phones'].isnull())) & ((joel.duplicated(subset=['Contact'],keep=False)))]

In [121]:
pair = nodupes_phone_hom.groupby(nodupes_phone_hom['Contact'].tolist(),as_index=False).size() ==3
lista = list(pair[pair==True].index)
nodupes_phone_hom = nodupes_phone_hom[nodupes_phone_hom['Contact'].apply(pairs) == True]

In [122]:
## Moving those Phones From 'Home Phone' Field to 'Home Phone 2' field
nodupes_phone_hom['Home Phone 2'] = nodupes_phone_hom['Home Phone'][nodupes_phone_hom.duplicated(subset=
['Contact'], keep=False)]
## Moving those Phones From 'Home Phone 2' Field to 'Other Phone' field
nodupes_phone_hom['Other Phone'] = nodupes_phone_hom['Home Phone 2']
#sorting:
nodupes_phone_hom['count'] = pd.isnull(nodupes_phone_hom[imp2]).sum(1)
nodupes_phone_hom= nodupes_phone_hom.sort_values(['Contact','count','Home Phone'],na_position='last')
nodupes_phone_hom.drop('count',1,inplace=True)
#Shifting Values:
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_phone_hom['Home Phone 2'] = nodupes_phone_hom.groupby(['Contact'])['Home Phone 2'].shift(-1)
nodupes_phone_hom['Other Phone'] = nodupes_phone_hom.groupby(['Contact'])['Other Phone'].shift(-2)

In [123]:
#Test - Before:
joel[joel.Contact==' Maia'][['Contact','Home Phone','Home Phone 2','Other Phone','E-mail Address','E-mail 2 Address',
                   'E-mail 3 Address']]

,Contact,Home Phone,Home Phone 2,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address
2603,Maia,+1 (508) 229-8219,NaN,NaN,paulomercatto@terra.com.br,leonardo.maia@cibt.com,NaN
3107,Maia,800992428,NaN,NaN,leonardo.maia@cibt.com,NaN,NaN
3802,Maia,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# joel Left JOIN nodupes_phone_hom
joel.update(nodupes_phone_hom, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [125]:
#Test - After:
joel[joel.Contact==' Maia'][['Contact','Home Phone','Home Phone 2','Other Phone','E-mail Address','E-mail 2 Address',
                   'E-mail 3 Address']]

,Contact,Home Phone,Home Phone 2,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address
2603,Maia,+1 (508) 229-8219,800992428,NaN,paulomercatto@terra.com.br,leonardo.maia@cibt.com,NaN
3107,Maia,800992428,NaN,NaN,leonardo.maia@cibt.com,NaN,NaN
3802,Maia,NaN,NaN,NaN,NaN,NaN,NaN


___

#### Mobile Phone to Other Phone and Business Phone 2

In [126]:
nodupes_phone_other = joel[((~joel.duplicated(subset=['Other Phones'], keep='last')) | (~joel.duplicated(subset=['Other Phones'], keep='first')) | (joel['Other Phones'].isnull())) & ((joel.duplicated(subset=['Contact'],keep=False)))]

In [127]:
pair = nodupes_phone_other.groupby(nodupes_phone_other['Contact'].tolist(),as_index=False).size() ==3
lista = list(pair[pair==True].index)
nodupes_phone_other = nodupes_phone_other[nodupes_phone_other['Contact'].apply(pairs) == True]

In [128]:
## Moving those Phones From 'Mobile Phone' Field to 'Other Phone' field
nodupes_phone_other['Other Phone'] = nodupes_phone_other['Mobile Phone'][nodupes_phone_other.duplicated(subset=
['Contact'], keep=False)]
## Moving those Phones From 'Other Phone' Field to 'Business Phone 2' field
nodupes_phone_other['Business Phone 2'] = nodupes_phone_other['Other Phone']
#sorting:
nodupes_phone_other['count'] = pd.isnull(nodupes_phone_other[imp2]).sum(1)
nodupes_phone_other= nodupes_phone_other.sort_values(['Contact','count','Mobile Phone'],na_position='last')
nodupes_phone_other.drop('count',1,inplace=True)
#Shifting Values:
### DISCLAIMER: MUST RUN THE FOLLOWING CODE ONLY ONCE!!
nodupes_phone_other['Other Phone'] = nodupes_phone_other.groupby(['Contact'])['Other Phone'].shift(-1)
nodupes_phone_other['Business Phone 2'] = nodupes_phone_other.groupby(['Contact'])['Business Phone 2'].shift(-2)

In [129]:
# Test: Before
joel[joel['Contact'].isin(['Valentin Samorukov'])][['Contact','Mobile Phone'
                ,'Other Phone','Business Phone 2']].sort_values('Contact')

,Contact,Mobile Phone,Other Phone,Business Phone 2
146,Valentin Samorukov,011 7 916 801-19-10,NaN,NaN
5658,Valentin Samorukov,+1 954-274-3671,NaN,NaN
6706,Valentin Samorukov,(786) 201-6169,NaN,NaN


#### Joining back to original Data set and deleting less useful rows:

In [130]:
# joel Left JOIN nodupes_phone_bus
joel.update(nodupes_phone_other, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [131]:
# Test: After
joel[joel['Contact'].isin(['Valentin Samorukov'])][['Contact','Mobile Phone'
                ,'Other Phone','Business Phone 2','E-mail Address','E-mail 2 Address',
                                                   'E-mail 3 Address']].sort_values('Contact')

,Contact,Mobile Phone,Other Phone,Business Phone 2,E-mail Address,E-mail 2 Address,E-mail 3 Address
146,Valentin Samorukov,011 7 916 801-19-10,NaN,NaN,val.samorukov@gmail.com,NaN,NaN
5658,Valentin Samorukov,+1 954-274-3671,011 7 916 801-19-10,NaN,NaN,NaN,NaN
6706,Valentin Samorukov,(786) 201-6169,+1 954-274-3671,011 7 916 801-19-10,samorucorp@mail.ru,val.samorukov@gmail.com,NaN


---

### 4.3 Excluding Records with duplicated names (pairs and trios):

##### Until now, we haven't deleted a single duplicate record. However, we will be doing this in the upcoming section.                                                   In order to prevent data loss, we will be using our well-known methodology which counts the number of blank fields for each records. Sorting the records by this counter in ascending order will prove to be conveninent after grouping the duplicates as we end up with the most filled records as the first occurence inside the group. After that, we will proceed by excluding the ones underneath the first occurence.

In [132]:
## Finding once again records with duplicated contact name:

dupes = joel[(joel.duplicated(subset=['Contact'], keep=False))]                                                 

In [133]:
# Function to identify pairs and trios of duplicated records

pair = dupes.groupby(dupes['Contact'].tolist(),as_index=False).size() <4
lista = list(pair[pair==True].index)
dupes = dupes[dupes['Contact'].apply(pairs) == True]

###### We can see that out of the 826 records on this subset, only 384 are unique in their name. That means that more than half of the records are somewhat redundant!

In [134]:
len(dupes)

826

In [135]:
dupes['Contact'].nunique()

384

In [136]:
dupes['count'] = pd.isnull(dupes[imp2]).sum(1)
dupes = dupes.sort_values(['count'])

##### But before we delete our records, We first need to merge all notes from each record group into that first, more-filled observation. 

In [137]:
dupes.reset_index(inplace=True)

a= dupes.groupby('Contact',as_index=False).agg(dicts)
                    
a.set_index('index',inplace=True)

In [138]:
dupes.set_index('index',inplace=True)

In [139]:
# dupes Left JOIN a
dupes.update(a, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [140]:
dupes = dupes.applymap(lambda x: np.nan if (isinstance(x,str) and special_match(x) == True) else x)

In [141]:
# Remove repeated character pattern in a string (for preventing redundant info after the aggregation)
import re
dupes[join_these] = dupes[join_these].applymap(lambda x: re.sub(r'(.+?)\1+', r'\1', x) if isinstance(x,str)  else x)

In [142]:
# Test -Before-:
joel[joel['Contact']=='Miguel de Oliveira'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
24,Miguel de Oliveira,NaN,NaN,NaN,NaN,(407) 467-8843,01155966451901,maoliveira@earthlink.net,miguel1956@uol.com.br,NaN,Amigo de Joey Volpato\r\n\r\nCCBTV\r\n\r\nNot ...
25,Miguel de Oliveira,NaN,NaN,NaN,NaN,01155966451901,NaN,miguel1956@uol.com.br,NaN,NaN,Olivia Ilmar Dantas da Silva Oliveira\n\nVeiu ...


In [143]:
# Test -Before-:
joel[joel['Contact']==' Alcantara'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
2403,Alcantara,NaN,NaN,NaN,NaN,NaN,NaN,tony@commtogo.com,wellingtonalc@hotmaiol.com,NaN,"Sunday, September 15, 2013 8:07 PM:\nWednesday..."
4624,Alcantara,NaN,NaN,NaN,NaN,(904) 759-0478,NaN,wellingtonalc@hotmaiol.com,NaN,NaN,NaN


In [144]:
# joel Left JOIN dupes
joel.update(dupes, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [145]:
#drop the duplicates using column 'Contact' and 'E-mail' as reference but only keep the most filled rows
dupes_2= dupes.drop_duplicates(subset=['Contact'],keep='first')

joel.drop(dupes.index.difference(dupes_2.index),inplace=True)

In [146]:
# Test -After-
joel[joel['Contact']=='Miguel de Oliveira'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
24,Miguel de Oliveira,NaN,NaN,NaN,NaN,(407) 467-8843,01155966451901,maoliveira@earthlink.net,miguel1956@uol.com.br,NaN,Amigo de Joey Volpato\r\n\r\nCBTV\r\n\r\nNot o...


In [147]:
joel.loc[joel[joel['Contact']=='Miguel de Oliveira'].index.item()]['Notes']

'Amigo de Joey Volpato\r\n\r\nCBTV\r\n\r\nNot on Linked In\r\n\r\nCalvario City Church\r\nCcorl.org\r\nPastor Miguel & Angelica de Oliveira\r\nIntercesion Pastors, Portuguese Ministry\r\n250 W Oak Ridge Road\r\nOrlando, FL 32809\r\n\r\n\r\n | Olivia Ilmar Dantas da Silva Oliveira\n\nVeiu de Orlando\n\nFilho de 26 anos de idade. \n\nFilho quer visto, foi recusado 3 vezes, mexa com venda compra de caros, pais agradáveis, talvez querem investir 50 mil, pode ser em nome do filho?\n'

In [148]:
# Test -After-:
joel[joel['Contact']==' Alcantara'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
2403,Alcantara,NaN,NaN,NaN,NaN,(904) 759-0478,NaN,tony@commtogo.com,wellingtonalc@hotmaiol.com,NaN,"Sunday, September 15, 2013 8:07 PM:\nWednesday..."


___

##### &#9678; Current Number of Records:

In [149]:
len(joel.index)

9074

---

## 5. Data on wrong Fields:

##### In this section, we will be moving information that somehow got mistakenly inputted into another field to the one that it is supposed to be.

##### Functions for finding non-alphabetical data on supposedly alphabetical fields and non-numeric data on supposedly Numerical fields 

In [150]:
## Function for finding if string is numerical or not:
def numeric(x):
    if pd.isna(x) == False:
        x = x.replace(' ','').replace('+','').replace('(','').replace(')','').replace('-','')
        return x.isnumeric()


## Function for finding if string is aplhabetical or not
def alpha(x):
    if pd.isna(x) == False:
        x= x.replace(".","").replace(' ',"").replace("'","").replace("`","")
        x= x.replace("-","").replace('?',"").replace('(','').replace(')','').replace('"','')
        x= x.replace(':','').replace('&','').replace(",",'').replace('/','')
        return x.isalpha()

### 5.1 Non-alphabetical Data on 'Contact'

In [151]:
#Non alphabetical on 'Contact'
nonalpha = joel[joel['Contact'].apply(alpha) == False]

imp = ['Contact','Business Phone','Business Phone 2','Home Phone','Home Phone 2','Mobile Phone','Other Phone',
 'E-mail Address','E-mail 2 Address']

In [152]:
len(nonalpha)

116

In [153]:
## Filtering only emails on 'Contact' field
ContactMail = joel[joel['Contact'].str.contains("@")==True]
len(ContactMail)

40

In [154]:
## Moving those mails From 'Contact' Field to 'E-mail 2 Address' field
ContactMail.loc[:]['E-mail 2 Address'] = ContactMail['Contact']

In [155]:
ContactMail[imp2].head(20)

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
164,webmaster@aa.com,NaN,NaN,NaN,NaN,NaN,NaN,webmaster@aa.com,webmaster@aa.com,NaN,
175,ts@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,ts@gmail.com,ts@gmail.com,NaN,
195,support@bberry.com,NaN,NaN,NaN,NaN,NaN,NaN,support@bberry.com,support@bberry.com,NaN,
219,sarah.regensdor@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,sarah.regensdor@gmail.com,sarah.regensdor@gmail.com,NaN,
260,pbockweg@bellsouth.net,NaN,NaN,NaN,NaN,NaN,NaN,pbockweg@bellsouth.net,pbockweg@bellsouth.net,NaN,
290,mlh@hotmail.com,NaN,NaN,NaN,NaN,NaN,NaN,mlh@hotmail.com,mlh@hotmail.com,NaN,
430,joel@jslaw.com,NaN,NaN,NaN,NaN,NaN,NaN,joel@jslaw.com,joel@jslaw.com,NaN,
436,joao_c_vicente@hotmail.com,NaN,NaN,NaN,NaN,NaN,NaN,joao_c_vicente@hotmail.com,joao_c_vicente@hotmail.com,NaN,
493,help@mgo.com,NaN,NaN,NaN,NaN,NaN,NaN,help@mgo.com,help@mgo.com,NaN,
494,hartism@comcast.net,NaN,NaN,NaN,NaN,NaN,NaN,hartism@comcast.net,hartism@comcast.net,NaN,


In [156]:
# act Left JOIN PhoneMail
joel.update(ContactMail, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [157]:
# Excluding Contacts that don't have at least one Phone Number:
ContactMail_2 = ContactMail.drop(ContactMail.loc[(ContactMail['Business Phones'].isnull()) & (ContactMail['Home Phones'].isnull()) &(ContactMail['Other Phones'].isnull())].index)

In [158]:
joel.drop(ContactMail.index.difference(ContactMail_2.index),inplace=True)

###### Go over these with client and see the best way to deal with those Records

In [159]:
nonalpha = joel[joel['Contact'].apply(alpha) == False]
nonalpha[imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
8,5466331116904942 12/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Js.miami@gmail.com<mailto:Js.miami@gmail.com <...
42,Marcia Godinho954-468-7866,NaN,NaN,NaN,NaN,9544687866,NaN,NaN,NaN,NaN,Paralegal
124,Bibhor «Bob ' Upreti,9492990559,NaN,NaN,NaN,NaN,NaN,bob@eb5investors.com,NaN,NaN,NaN
151,????? ??????,NaN,NaN,NaN,NaN,NaN,NaN,joelstewart@rambler.ru,NaN,NaN,
202,Stanley Roth 2 Mindy,NaN,NaN,631-752-9696,NaN,NaN,NaN,NaN,NaN,NaN,
313,Mark_y Morais,NaN,NaN,NaN,NaN,17862223692,NaN,NaN,NaN,NaN,
338,Manuel + Cora Ejercito,NaN,NaN,NaN,NaN,NaN,NaN,coraejercito@gmail.com,NaN,NaN,
396,Jérôme Jérôme],NaN,NaN,NaN,NaN,NaN,NaN,famille.jackson@free.fr,NaN,NaN,
624,Ariane Ariane],NaN,NaN,NaN,NaN,NaN,NaN,arianebruneton@orange.fr,NaN,NaN,
841,RÃSOUF,NaN,NaN,NaN,NaN,NaN,NaN,louis.resouf@hec.ca,NaN,NaN,NaN


##### Fixing the encoding:

In [160]:
def decoder(name):
    if name[0]==' ':
        name = name.replace(' ','',1)       
    else:
        name = name
    name= name.encode('Latin-1',errors='ignore').decode('utf-8',errors='ignore')
    return name
    
    

In [161]:
nonalpha.loc[:]['Contact'] = nonalpha['Contact'].apply(decoder)

In [162]:
# act Left JOIN nonalpha
joel.update(nonalpha, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [163]:
nonalpha = joel[joel['Contact'].apply(alpha) == False]

###### Some Names needed to be decoded twice...

In [164]:
nonalpha[imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
8,5466331116904942 12/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Js.miami@gmail.com<mailto:Js.miami@gmail.com <...
42,Marcia Godinho954-468-7866,NaN,NaN,NaN,NaN,9544687866,NaN,NaN,NaN,NaN,Paralegal
151,????? ??????,NaN,NaN,NaN,NaN,NaN,NaN,joelstewart@rambler.ru,NaN,NaN,
202,Stanley Roth 2 Mindy,NaN,NaN,631-752-9696,NaN,NaN,NaN,NaN,NaN,NaN,
313,Mark_y Morais,NaN,NaN,NaN,NaN,17862223692,NaN,NaN,NaN,NaN,
338,Manuel + Cora Ejercito,NaN,NaN,NaN,NaN,NaN,NaN,coraejercito@gmail.com,NaN,NaN,
396,Jrme Jrme],NaN,NaN,NaN,NaN,NaN,NaN,famille.jackson@free.fr,NaN,NaN,
624,Ariane Ariane],NaN,NaN,NaN,NaN,NaN,NaN,arianebruneton@orange.fr,NaN,NaN,
1144,DÃ¡ngelo,NaN,NaN,+1 (305) 651-9311,NaN,NaN,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 7:21 PM:\nGoffredo ..."
1552,,NaN,NaN,NaN,NaN,+1 (2o3)536-9169,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 8:11 PM:\n<!DOCTYPE..."


In [165]:
# Example -After-:
nonalpha[nonalpha['Home Phone']=='+1 (305) 651-9311']['Contact']

1144    DÃ¡ngelo
Name: Contact, dtype: object

In [166]:
def decoder2(name):
    name= name.encode('Latin-1').decode('utf-8',errors='ignore')
    return name
    

In [167]:
nonalpha.loc[:]['Contact'] = nonalpha['Contact'].apply(decoder2)

In [168]:
# Example -After-:
nonalpha[nonalpha['Home Phone']=='+1 (305) 651-9311']['Contact']

1144    Dángelo
Name: Contact, dtype: object

In [169]:
# act Left JOIN nonalpha
joel.update(nonalpha, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [170]:
## Test -After-:
joel.loc[[1139,895,2595,3419,2257,2639,7536,8959,1144],'Contact']

1139    da Conceição Gomes
895                Brandão
2595               Rancaño
3419                 Núñez
2257       da Rocha Galvão
2639             Саморуков
7536         Noué Isabelle
8959               Евгений
1144               Dángelo
Name: Contact, dtype: object

___

## 6  Duplicate Exceptions

###### We've successfully taken care of the fully redundant records in section 4. We then proceeded by dealing with doubles and trios of almost-duplicated rows in subsection 4.1 , 4.2 and 4.3. 

###### At last, we are now going to be dealing with the almost-duplicate records which surpassed the trio status regarding their number of instances inside their 'Contact' name group.

##### &#9678; Current Number of Duplicates on Contact

In [171]:
len(joel[joel.duplicated(subset=['Contact'],keep=False)])

104

___

In [172]:
imp = ['Contact',
 'Business Phone',
 'Business Phone 2',
 'Home Phone',
    'Home Phone 2',
 'Mobile Phone',
       'Other Phone',
 'E-mail Address',
 'E-mail 2 Address',
      'count']

In [173]:
remaining = joel[joel.duplicated(subset=['Contact'],keep=False)].sort_values('Contact')

In [174]:
remaining = joel[joel.duplicated(subset=['Contact'],keep=False)].sort_values('Contact')

# Let's get our 'count' in place once again:

remaining['count'] = pd.isnull(remaining).sum(1)

###### The next sets of duplicated contacts will have its relevant information joined/aggregated into the first occurrence (most filled one). note that we are not able to 'spread' the data between multiple fields simply because, in this particular case, Outlook doesn't offer that many fields.

In [175]:
save =remaining
save['count'] = pd.isnull(save).sum(1)
save = save.sort_values(['Contact','count'])


In [176]:
dicts = {}
value_1 = lambda x: '| '.join(x.fillna('').astype(str))

for i in firsts + join_these:
    dicts[i] = value_1

In [177]:
save.reset_index(inplace=True)

dicts['index'] = 'first'

# Aggregation:

a= save.groupby('Contact',as_index=False).agg(dicts)
                    
a.set_index('index',inplace=True)

In [178]:
save.set_index('index',inplace=True)

In [179]:
# save Left JOIN a
save.update(a, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [180]:
save = save.applymap(lambda x: np.nan if (isinstance(x,str) and special_match(x) == True) else x)

In [181]:
# Remove repeated character pattern in a string (for preventing redundant info after the aggregation)

import re
save[join_these] = save[join_these].applymap(lambda x: re.sub(r'(.+?)\1+', r'\1', x) if isinstance(x,str)  else x)

In [182]:
## Example - Before - 
# (Note that it would be impossible to distribute all emails accordingly).:
joel[joel['Contact']==' Carvalho'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
1473,Carvalho,(954) 782-0430,NaN,+1 (954) 788-2391,NaN,NaN,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 8:09 PM:\n<!DOCTYPE..."
1741,Carvalho,(781) 856-7172,NaN,+1 (781) 856-7172,NaN,(617) 389-1879,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 7:20 PM:\nCarvalho,..."
1978,Carvalho,(305) 379-7999,NaN,(180) 038-7275,NaN,NaN,NaN,info@gosouth.com,info@remessa.com,NaN,"Sunday, September 15, 2013 8:14 PM:\n<!DOCTYPE..."
2934,Carvalho,(305) 960-8426,NaN,NaN,NaN,(786) 301-3020,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 8:09 PM:\n<!DOCTYPE..."
3815,Carvalho,(407) 370-5086,NaN,NaN,NaN,(407) 953-3479,NaN,rodrigo@planetuptechnology.com,NaN,NaN,"Sunday, September 15, 2013 7:20 PM:\nRodrigo C..."
5233,Carvalho,(786) 602-9114,NaN,NaN,NaN,NaN,NaN,gabriel@eyeseasolutions.com,NaN,NaN,NaN
5269,Carvalho,NaN,NaN,+1 (954) 601-6575,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
# joel Left JOIN dupes
joel.update(save, join='left', overwrite=True, filter_func=None, raise_conflict=False)

In [184]:
#drop the duplicates using column 'Contact' and 'E-mail' as reference but only keep the most filled rows
save_2= save.drop_duplicates(subset=['Contact'],keep='first')

joel.drop(save.index.difference(save_2.index),inplace=True)

In [186]:
## Example - After - :
joel[joel['Contact']==' Carvalho'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
1978,Carvalho,(305) 379-7999| (781) 856-7172| (305) 960-8426...,NaN,(180) 038-7275| +1 (781) 856-7172| | | +1 (954...,NaN,| (617) 389-1879| (786) 301-3020| | | (407) 95...,NaN,info@gosouth.com| | | gabriel@eyeseasolutions....,info@remessa.com| | | | | |,NaN,"Sunday, September 15, 2013 8:14 PM:\n<!DOCTYPE..."


____

##### &#9678; Final Number of Duplicates on 'Contact'

In [187]:
len(joel[joel.duplicated(subset=['Contact'],keep=False)])

0

##### &#9678;  Final Number of Records

In [188]:
len(joel)

8962

____

# Results!

### Lets recap on all the examples shown on this project in a before/after fashion.

#### joel_0 will represent the original state of the records

In [206]:
joel_0 = pd.read_csv("Joel_hotmail.csv",encoding='Latin-1',dtype='object')

In [207]:
joel_0["Contact"] = joel_0["First Name"].fillna('') +' '+ joel_0["Last Name"].fillna('')

In [208]:
## Example -Before- :
joel_0[joel_0['Contact']=='Tatiana Goncalves'][imp2].head()

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
721,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
722,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
723,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
724,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n
725,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\r\n\r\n


In [209]:
## Example -After- :
joel[joel['Contact']=='Tatiana Goncalves'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
12,Tatiana Goncalves,NaN,NaN,NaN,NaN,(754) 368-0417,NaN,taty.goncalves@hotmail.com,NaN,NaN,Works with\n\n Works with Urbano.\n\n| Works ...


____

In [210]:
## Example -Before- :
joel_0[joel_0.Contact==' Adams'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
3748,Adams,(513) 977-8125,NaN,+1 (513) 651-6489,NaN,NaN,+1 (513) 651-6981,gregory.adams@dinslaw.com,NaN,NaN,"Sunday, September 15, 2013 8:07 PM:\nWednesday..."
5066,Adams,(561) 272-8037,NaN,NaN,NaN,+1 (561) 654-6288,NaN,mattadams97@hotmail.com,NaN,NaN,"Sunday, September 15, 2013 7:19 PM:\nMatthew D..."


In [211]:
## Example -After- :
joel[joel.Contact==' Adams'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes


___

In [212]:
#test -Before-:
joel_0[joel_0['Contact']=='Miguel de Oliveira'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
24,Miguel de Oliveira,NaN,NaN,NaN,NaN,(407) 467-8843,NaN,maoliveira@earthlink.net,NaN,NaN,Amigo de Joey Volpato\r\n\r\nCCBTV\r\n\r\nNot ...
25,Miguel de Oliveira,NaN,NaN,NaN,NaN,01155966451901,NaN,miguel1956@uol.com.br,NaN,NaN,Olivia Ilmar Dantas da Silva Oliveira\n\nVeiu ...


In [213]:
#test -After-:
joel[joel['Contact']=='Miguel de Oliveira'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
11927,Miguel de Oliveira,NaN,NaN,NaN,NaN,(407) 467-8843,NaN,maoliveira@earthlink.net,NaN,NaN,Amigo de Joey Volpato\n\nCCBTV\n\nNot on Linke...


___

In [214]:
#test -Before-:
joel_0[joel_0['Contact']=='Selma Smith'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
8758,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,Alohaselma@yahoo.com,NaN,NaN,NaN
10342,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,selfiorini@msn.com,NaN,NaN,NaN
11465,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,selsmith66@yahoo.com,NaN,NaN,NaN


In [215]:
#test -After-:
joel[joel['Contact']=='Selma Smith'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
8056,Selma Smith,NaN,NaN,NaN,NaN,NaN,NaN,Alohaselma@yahoo.com,selfiorini@msn.com,selsmith66@yahoo.com,NaN


___

In [235]:
#Example -Before-:
joel_0[joel_0['Contact']==' Bezerra'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
2604,Bezerra,(727) 216-7979,NaN,+1 (727) 812-8874,NaN,NaN,NaN,zenilton@tbrazilianpost.com,NaN,NaN,"Sunday, September 15, 2013 8:14 PM:\n<!DOCTYPE..."
3566,Bezerra,(305) 254-6672,NaN,(305) 254-6672,+1 (305) 254-6672,(305) 281-8691,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 8:09 PM:\n<!DOCTYPE..."


In [107]:
#Example -After-:
joel[joel['Contact']==' Bezerra'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
2604,Bezerra,(727) 216-7979,NaN,+1 (727) 812-8874,NaN,NaN,NaN,zenilton@tbrazilianpost.com,NaN,NaN,"Sunday, September 15, 2013 8:14 PM:\n<!DOCTYPE..."
3566,Bezerra,(305) 254-6672,NaN,(305) 254-6672,+1 (305) 254-6672,(305) 281-8691,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 8:09 PM:\n<!DOCTYPE..."


___

In [218]:
#Example -Before-:
joel_0[joel_0['Contact']=='Victor Hugo'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
5427,Victor Hugo,NaN,NaN,4075484101,NaN,NaN,NaN,NaN,NaN,NaN,Colega Guilherme.
6055,Victor Hugo,NaN,NaN,+55 24 99925-5410,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
#Example -After-:
joel[joel['Contact']=='Victor Hugo'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
4715,Victor Hugo,NaN,NaN,4075484101,+55 24 99925-5410,NaN,NaN,NaN,NaN,NaN,Colega Guilherme.


___

In [220]:
#Example -Before-:
joel_0[joel_0['Contact']=='Maria Abreu'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
706,Maria Abreu,NaN,NaN,NaN,NaN,(786) 599-7994,NaN,DAbreu@globalgbi.com,NaN,NaN,"My address is: 11503 NW 89th St, Unit 109, Dor..."
7685,Maria Abreu,NaN,NaN,NaN,NaN,+1 (407) 963-8598,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 7:19 PM:\nMaria Ang..."


In [221]:
#Example -After-:
joel[joel['Contact']=='Maria Abreu'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes


In [222]:
joel.loc[706,'Notes']

'Sunday, September 15, 2013 7:50 PM:\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2//EN">\n<HTML>\n<HEAD>\n<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">\n<META NAME="Generator" CONTENT="MS Exchange Server version rmj.rmm.rup.rpr">\n<TITL'

____

In [223]:
#test -Before-:
joel_0[joel_0['Contact']==' Arruda'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
2167,Arruda,(305) 801-9035,NaN,(786) 521-2038,NaN,NaN,NaN,arruda@itamaraty.gov.br,NaN,fernando.arruda@gmail.com,NaN
2642,Arruda,(305) 285-6222,NaN,(786) 521-2038,NaN,(786) 350-5791,NaN,fernando.arruda@itamaraty.gov.br,Fernando.Arruda@itamaraty.gov.br,NaN,"Sunday, September 15, 2013 7:50 PM:\n\n-------..."
4872,Arruda,21 21047256,NaN,NaN,NaN,NaN,NaN,u27comandante@yahoo.com.br,NaN,NaN,NaN


In [224]:
#test -After-:
joel[joel['Contact']==' Arruda'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
1926,Arruda,(305) 285-6222,(305) 801-9035,(786) 521-2038,NaN,(786) 350-5791,NaN,fernando.arruda@itamaraty.gov.br,arruda@itamaraty.gov.br,fernando.arruda@gmail.com,"Sunday, September 15, 2013 7:50 PM:\n\n-------..."


___

In [225]:
# Test: Before
joel_0[joel_0['Contact'].isin(['Valentin Samorukov'])][imp2].sort_values('Contact')

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
146,Valentin Samorukov,NaN,NaN,NaN,NaN,011 7 916 801-19-10,NaN,val.samorukov@gmail.com,NaN,NaN,NaN
5658,Valentin Samorukov,NaN,NaN,NaN,NaN,+1 954-274-3671,NaN,NaN,NaN,NaN,Nadezhda
6706,Valentin Samorukov,NaN,NaN,NaN,NaN,(786) 201-6169,NaN,samorucorp@mail.ru,NaN,NaN,NaN


In [226]:
# Test: After
joel[joel['Contact'].isin(['Valentin Samorukov'])][imp2].sort_values('Contact')

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
5998,Valentin Samorukov,NaN,011 7 916 801-19-10,NaN,NaN,(786) 201-6169,+1 954-274-3671,samorucorp@mail.ru,val.samorukov@gmail.com,NaN,Nadezhda|


____

In [227]:
# Test -Before-:
joel_0[joel_0['Contact']==' Alcantara'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
2403,Alcantara,NaN,NaN,NaN,NaN,NaN,NaN,tony@commtogo.com,NaN,NaN,"Sunday, September 15, 2013 8:07 PM:\nWednesday..."
4624,Alcantara,NaN,NaN,NaN,NaN,(904) 759-0478,NaN,wellingtonalc@hotmaiol.com,NaN,NaN,NaN


In [228]:
# Test -After-:
joel[joel['Contact']==' Alcantara'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
1687,Alcantara,NaN,NaN,NaN,NaN,(904) 759-0478,NaN,tony@commtogo.com,wellingtonalc@hotmaiol.com,NaN,"Sunday, September 15, 2013 8:07 PM:\nWednesday..."


___

In [229]:
## Example - Before - 
# (Note that it would be impossible to distribute all phone numbers accordingly).:
joel_0.loc[joel_0['Contact']==' Ramos'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
1339,Ramos,NaN,(914) 924-0125,NaN,+1 (914) 771-8824,+1 (914) 497-4813,NaN,NaN,NaN,NaN,"Sunday, September 15, 2013 7:28 PM:\nDulcimar ..."
4253,Ramos,NaN,NaN,+1 (561) 504-3428,NaN,+1 (561) 504-5354,+1 (305) 328-4087,mirita28@hotmail.com,NaN,NaN,"Sunday, September 15, 2013 7:28 PM:\nMiriam Ra..."
4608,Ramos,(407) 999-0364,NaN,NaN,NaN,(407) 421-8841,NaN,eramos@ft.newyorklife.com,NaN,NaN,conselho
5272,Ramos,NaN,NaN,(407) 288-2875,NaN,NaN,NaN,anderson@frinvrstgroup.com,NaN,NaN,NaN


In [232]:
## Example - Before - 
joel.loc[joel['Contact']==' Ramos'][imp2]

,Contact,Business Phone,Business Phone 2,Home Phone,Home Phone 2,Mobile Phone,Other Phone,E-mail Address,E-mail 2 Address,E-mail 3 Address,Notes
3536,Ramos,(914) 924-0125,NaN,+1 (561) 504-3428,+1 (914) 771-8824,+1 (561) 504-5354,+1 (914) 497-4813,mirita28@hotmail.com,anderson@frinvrstgroup.com,NaN,"Sunday, September 15, 2013 7:28 PM:\nMiriam Ra..."


___

#### Eliminating Created custom columns:

In [233]:
joel.drop(['Contact','Home Phones','Business Phones', 'Other Phones','E-mail Addresses'],1,inplace=True)

In [234]:
joel.columns.values

array(['Unnamed: 0', 'Title', 'First Name', 'Middle Name', 'Last Name',
       'Suffix', 'Company', 'Department', 'Job Title', 'Business Street',
       'Business Street 2', 'Business Street 3', 'Business City',
       'Business State', 'Business Postal Code',
       'Business Country/Region', 'Home Street', 'Home Street 2',
       'Home Street 3', 'Home City', 'Home State', 'Home Postal Code',
       'Home Country/Region', 'Other Street', 'Other Street 2',
       'Other Street 3', 'Other City', 'Other State', 'Other Postal Code',
       'Other Country/Region', "Assistant's Phone", 'Business Fax',
       'Business Phone', 'Business Phone 2', 'Callback', 'Car Phone',
       'Company Main Phone', 'Home Fax', 'Home Phone', 'Home Phone 2',
       'ISDN', 'Mobile Phone', 'Other Fax', 'Other Phone', 'Pager',
       'Primary Phone', 'Radio Phone', 'TTY/TDD Phone', 'Telex',
       'Account', 'Anniversary', "Assistant's Name",
       'Billing Information', 'Birthday', 'Business Address PO Box',

___

# Exporting

In [189]:
len(joel)

8962

In [238]:
writer = pd.ExcelWriter('Joel_dec212018.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})

In [239]:
joel.to_excel(writer,'Sheet1')

In [240]:
writer.save()